# JDBC - Java DataBase Connectivity

Es una API que permite la ejecución de operaciones sobre bases de datos desde el lenguaje de programación Java, independientemente del sistema operativo donde se ejecute o de la base de datos a la cual se accede, utilizando el dialecto SQL del modelo de base de datos que se utilice.

## API JDBC
| Clase | Descripción |
|--|--|
| java.sql.Connection | Representa una conexión a la base de datos. Abstrae el detalle de cómo comunicarse con el servidor de base de datos. |
| java.sql.DriverManager | Administra los drivers JDBC usados por la aplicación. En conjunto con el URL y la autenticación apropiada, puede proveer aplicaciones con objetos Connection válidos. |
|javax.sql.DataSource | Abstrae los detalles (URL, autenticación) de cómo obtener una conexión a la base de datos. Es el método preferido para obtener objetos Connection. |
| java.sql.Statement | Provee métodos para que los desarrolladores puedan ejecutar sentencias SQL. |
| java.sql.ResultSet | Representa los resultados de una sentencia SQL. Estos objetos son retornados usualmente por métodos del objeto Statement. |


## java.sql.DriverManager

Permite al desarrollador conseguir un objeto `Connection` la cual se puede usar para ejecutar actividades en la base de datos.

Para establecer la conexión a la base de datos, se debe usar el método `getConnection()` dándole el URL JDBC, también con el usuario y contraseña para acceder a la base de datos si aplica.

El URL debe seguir la sintaxis requerida por la implementación particular de la base de datos.

In [ ]:
var url = "jdbc:<SGBD>:<database>";
String user = "username";
String password = "password";
Connection conn = null;
try {
    conn = DriverManager.getConnection(url, user, password);
    ...
} catch (SQLException e) {
    // Manejo de los errores
}

## Cadenas de conexión JDBC
| DBMS | URL |
|--|--|
| IBM DB2 | `jdbc:db2://<HOST>:<PORT>/<DB>` |
| MySQL | `jdbc:mysql://<HOST>:<PORT>/<DB>` |
| Postgresql | `jdbc:postgresql://<HOST>:<PORT>/<DB>` |
| SQL Server | `jdbc:sqlserver://<HOST>\<DB>:<PORT>` |
| Oracle | `jdbc:oracle:thin:@<HOST>:<PORT>:<SID>` |
| JDBC-ODBC Bridge | `jdbc:odbc:<DB>` |
| **SQLite** | **`jdbc:sqlite:<FILE_PATH>`** |

## Conectando JDBC a SQLite

**pom.xml**
```xml
...
<dependencies>
    ...
    <dependency>
        <groupId>org.xerial</groupId>
        <artifactId>sqlite-jdbc</artifactId>
        <version>3.36.0.1</version>
    </dependency>
    ...
</dependencies>
...
```

Los objetos `java.sql.Connection` representa una conexión actual a la base de datos.

In [ ]:
var url = "jdbc:sqlite:sample.db";
Connection conn = null;
try {
    conn = DriverManager.getConnection(url);
} catch (SQLException e) {
    // Manejo de los errores
}

Una vez tenemos la instancia de este objeto, podemos crear una instancia de `java.sql.Statement`, el cual nos permitirá ejecutar sentencias SQL.

In [ ]:
var url = "jdbc:sqlite:sample.db";
Connection conn = null;
Statement stmt = null;
try {
    conn = DriverManager.getConnection(url);
    stmt = conn.createStatement();
} catch (SQLException e) {
    // Manejo de los errores
}

Los objetos `Statement` provee varios métodos para ejecutar SQL. Los dos más usados son:
- **executeQuery**: Toma una sentencia SELECT y retorna el resultado de la operación como un objeto ResultSet.
- **executeUpdate**: Toma una sentencia INSERT, UPDATE, o DELETE y retorna el número de filas afectadas por la operación.

In [ ]:
var url = "jdbc:sqlite:sample.db";
Connection conn = null;
Statement stmt = null;
ResultSet rs = null;
try {
    conn = DriverManager.getConnection(url);
    stmt = conn.createStatement();
    rs = stmt.executeQuery("SELECT * FROM employees");
    while (rs.next()) {
        int id = rs.getInt("employee_id");
        String nombre = rs.getString("first_name");
        String apellido = rs.getString("last_name");
        // Hacer algo con los datos leídos
    }
    rs.close();
} catch (SQLException e) {
    System.err.println("Un error mientras realiza la consulta " + e);
} finally {
    try {
        if (rs != null){
            rs.close();
        }
        if (stmt != null) {
            stmt.close();
        }
        if (conn != null){
            conn.close();
        }
    } catch (SQLException e) {
        System.err.println(e);
    }
}

Los datos dentro de un objeto `java.sql.ResultSet` pueden ser mejor visualizados como una tabla. La información puede ser recuperada de a una fila a la vez, y el objeto `ResultSet` mantendrá el control de la fila actual.

Para iterar por las filas, usamos el método next(), el
cual mueve el apuntador de fila a la siguiente. Este
método retorna true si existe una siguiente fila.
```Java
while (rs.next()) {
    //Lee los datos de la fila actual
}
```

Para obtener los datos de cada fila, el objeto `ResultSet` nos da varios métodos _get_ los cuales toman como parámetro el **indice** (empezando desde 1) de columna o el **nombre** de la columna.
- El método `getString()` es para obtener un dato String
- El método `getInt()` es para obtener un dato int
- El método `getBoolean()` es para obtener un dato boolean.
- etc.

Es recomendado, sin embargo, usar los nombres de los campos en lugar de la posición en la fila

### `java.sql.PreparedStatement`

Es una subclase de `java.sql.Statement` y contiene una consulta pre-compilada y puede estar esperando algún(os) parámetro(s).
```Java
var sql = "INSERT INTO GRADES (GRADE, MIN_SALARY, MAX_SALARY) VALUES ( ? , ? , ? )";
PreparedStatement pstmt = conn.prepareStatement(sql);
```
Para darle los valores a los parámetros que se definen en esta clase, usamos los métodos `setXXX()`, donde `XXX` es el tipo de dato que va a asignar. Los parámetros empiezan a contar desde 1.
```Java
pstmt.setString(1, "G");
pstmt.setLong(2, 1);
pstmt.setLong(3, 99999);
pstmt.executeUpdate();
```

 ### `java.sql.CallableStatement`

Subclase de `java.sql.PreparedStatement`, la cual va a ayudar a hacer las llamadas a funciones y procedimientos almacenados de la base de datos.
```Java
CallableStatement cstmt = conn.prepareCall("{ call myProcedure(?) }");
```

Al igual que el PreparedStatement, usamos los métodos `setXXX()` para asignar los parámetros. Los parámetros empiezan a contar desde 1.
```Java
cstmt.setString(1, "Hola");
cstmt.executeUpdate();
```

Si lo que queremos es hacer el llamado a una función almacenada que retorna un valor, entonces lo creamos:
```Java
cstmt = conn.prepareCall("{ ? = call myFunction(?) }");
```

Y debemos registrar qué parámetros serán de salida usando el método `registerOutParameter()`, antes de asignar valores a los parámetros y ejecutar el llamado.
```Java
cstmt.registerOutParameter(1, Type.INTEGER);
cstmt.setString(2, "Hola");
cstmt.excecuteUpdate();
```

Y para obtener el valor de los parámetros de salida, usamos el método `getXXX()`
```Java
int result = cstmt.getInt(1);
```
